## Seattle Terry Stops Model
This is the the third workbook for the Seattle Terry Stops project. In this part our objectives will be to:
* creating the best model that predicts the likelihood of an arrest during a Terry Stop


## Loading the data
Let us start by loading the data we will use for this process.

In [2]:
# Importing the relevant libraries and modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Loading the data
df = pd.read_csv('data/clean_Terry_stops_data.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_gender,officer_race,subject_perceived_race,...,repeat_offenders,incident_year,incident_month,officer_age,field_contact,offense_report,reported_hour,dif_race,dif_gender,dif_race_gender
0,1_17,unassigned,20150000084533,28020,Referred for Prosecution,Knife/Stabbing Instrument,4585,M,Hispanic,Black or African American,...,No,2015,3,60,n,n,16,True,True,Y
1,36_45,unassigned,20150000001428,28092,Field Contact,NaN,7634,M,White,Multi-Racial,...,No,2015,3,38,y,n,5,True,False,Y
2,18_25,unassigned,20150000001428,28093,Field Contact,NaN,7634,M,White,White,...,No,2015,3,38,y,n,5,False,False,N
3,26_35,unassigned,20150000001437,28381,Field Contact,NaN,7634,M,White,White,...,No,2015,3,38,y,n,10,False,False,N
4,36_45,unassigned,20150000087329,28462,Offense Report,NaN,7634,M,White,White,...,No,2015,3,38,n,y,11,False,False,N
